### 0.0. IMPORTS

In [4]:
#IMPORTANDO AS BIBLIOTECAS
import pandas as pd
import plotly.express as px
import folium
from folium.plugins import MarkerCluster
import inflection
import streamlit as st
from PIL import Image
import locale
from streamlit_folium import folium_static
import numpy as np


### ETL

### Extraction
        - Fonte: Arquivo CSV no Disco Rígido (HD)

### Transformation
        - Linguagem de Programação Python

### LOAD
        -  Dataframe em memória

### 0.1 Variáveis de Ajuda

In [5]:
DATA_PATH = f"data/old/zomato.csv"

COUNTRIES = {
    1: "India",
    14: "Australia",
    30: "Brazil",
    37: "Canada",
    94: "Indonesia",
    148: "New Zeland",
    162: "Philippines",
    166: "Qatar",
    184: "Singapure",
    189: "South Africa",
    191: "Sri Lanka",
    208: "Turkey",
    214: "United Arab Emirates",
    215: "England",
    216: "United States of America",
}


COLORS = {
    "3F7E00": "darkgreen",
    "5BA829": "green",
    "9ACD32": "lightgreen",
    "CDD614": "orange",
    "FFBA00": "red",
    "CBCBC8": "darkred",
    "FF7800": "darkred",
}

### 0.2. Funções de Ajuda

In [6]:
def rename_columns(dataframe):
    df = dataframe.copy()

    title = lambda x: inflection.titleize(x)

    snakecase = lambda x: inflection.underscore(x)

    spaces = lambda x: x.replace(" ", "")

    cols_old = list(df.columns)

    cols_old = list(map(title, cols_old))

    cols_old = list(map(spaces, cols_old))

    cols_new = list(map(snakecase, cols_old))

    df.columns = cols_new

    return df

In [7]:
def country_name(country_id):
    """ 
     1. Para colocar o nome dos países com base no código de cada país
        
        Preenchimento do nome dos países
    """
    return COUNTRIES[country_id]

In [8]:
def create_price_tye(price_range):
    """
    2. Criar a categoria do tipo de comida com base no range de valores.

        Criação do Tipo de Categoria de Comida
    """
    if price_range == 1:
        return "cheap"
    elif price_range == 2:
     
     return "normal"
    
    elif price_range == 3:
        return "expensive"
    
    else:
        return "gourmet"

In [9]:
def color_name(color_code):
    """
    3. Criar o nome das cores com base nos códigos de cores
        Criação do nome das Cores
    """
    return COLORS[color_code]

In [10]:
def adjust_columns_order(dataframe):
    df = dataframe.copy()

    new_cols_order = [
        "restaurant_id",
        "restaurant_name",
        "country_name",
        "city",
        "address",
        "locality",
        "locality_verbose",
        "longitude",
        "latitude",
        "cuisines",
        "price_type",
        "average_cost_for_two",
        "currency",
        "has_table_booking",
        "has_online_delivery",
        "is_delivering_now",
        "aggregate_rating",
        "rating_color",
        "color_name",
        "rating_text",
        "votes",
    ]

    return df.loc[:, new_cols_order]

In [11]:
def process_data(file_path):
    df = pd.read_csv(file_path)

    df = df.dropna()

    df = rename_columns(df)

    df["price_type"] = df.loc[:, "price_range"].apply(lambda x: create_price_tye(x))

    df["country_name"] = df.loc[:, "country_code"].apply(lambda x: country_name(x))

    df["color_name"] = df.loc[:, "rating_color"].apply(lambda x: color_name(x))

    df["cuisines"] = df.loc[:, "cuisines"].apply(lambda x: x.split(",")[0])

    df = df.drop_duplicates()

    df = adjust_columns_order(df)

    df.to_csv("data/processed/zomato.csv", index=False)

    return df



### 0.4. Load Data

In [12]:
df_raw = pd.read_csv(DATA_PATH)
df_raw.head()

,Restaurant ID,Restaurant Name,Country Code,City,Address,Locality,Locality Verbose,Longitude,Latitude,Cuisines,...,Currency,Has Table booking,Has Online delivery,Is delivering now,Switch to order menu,Price range,Aggregate rating,Rating color,Rating text,Votes
0,6310675,Mama Lou's Italian Kitchen,162,Las Piñas City,"Block 1, Lot 36, Tropical Avenue Corner Tropic...",BF International,"BF International, Las Piñas City",121.009787,14.447615,Italian,...,Botswana Pula(P),1,0,0,0,3,4.6,3F7E00,Excellent,619
1,6310675,Mama Lou's Italian Kitchen,162,Las Piñas City,"Block 1, Lot 36, Tropical Avenue Corner Tropic...",BF International,"BF International, Las Piñas City",121.009787,14.447615,Italian,...,Botswana Pula(P),1,0,0,0,3,4.6,3F7E00,Excellent,619
2,6314542,Blackbird,162,Makati City,"Nielson Tower, Ayala Triangle Gardens, Salcedo...","Ayala Triangle Gardens, Salcedo Village, Makat...","Ayala Triangle Gardens, Salcedo Village, Makat...",121.024562,14.556042,"European, Asian",...,Botswana Pula(P),0,0,0,0,4,4.7,3F7E00,Excellent,469
3,6301293,Banapple,162,Makati City,"Ayala Triangle Gardens, Salcedo Village, Makat...","Ayala Triangle Gardens, Salcedo Village, Makat...","Ayala Triangle Gardens, Salcedo Village, Makat...",121.023171,14.556196,"Filipino, American, Italian, Bakery",...,Botswana Pula(P),0,0,0,0,3,4.4,5BA829,Very Good,867
4,6315689,Bad Bird,162,Makati City,"Hole In The Wall, Floor 4, Century City Mall, ...","Century City Mall, Poblacion, Makati City","Century City Mall, Poblacion, Makati City, Mak...",121.027708,14.565899,American,...,Botswana Pula(P),0,0,0,0,3,4.4,5BA829,Very Good,858


## 1.0. Descrição de Dados

In [13]:
df2 = df_raw.copy()
df2.head()

,Restaurant ID,Restaurant Name,Country Code,City,Address,Locality,Locality Verbose,Longitude,Latitude,Cuisines,...,Currency,Has Table booking,Has Online delivery,Is delivering now,Switch to order menu,Price range,Aggregate rating,Rating color,Rating text,Votes
0,6310675,Mama Lou's Italian Kitchen,162,Las Piñas City,"Block 1, Lot 36, Tropical Avenue Corner Tropic...",BF International,"BF International, Las Piñas City",121.009787,14.447615,Italian,...,Botswana Pula(P),1,0,0,0,3,4.6,3F7E00,Excellent,619
1,6310675,Mama Lou's Italian Kitchen,162,Las Piñas City,"Block 1, Lot 36, Tropical Avenue Corner Tropic...",BF International,"BF International, Las Piñas City",121.009787,14.447615,Italian,...,Botswana Pula(P),1,0,0,0,3,4.6,3F7E00,Excellent,619
2,6314542,Blackbird,162,Makati City,"Nielson Tower, Ayala Triangle Gardens, Salcedo...","Ayala Triangle Gardens, Salcedo Village, Makat...","Ayala Triangle Gardens, Salcedo Village, Makat...",121.024562,14.556042,"European, Asian",...,Botswana Pula(P),0,0,0,0,4,4.7,3F7E00,Excellent,469
3,6301293,Banapple,162,Makati City,"Ayala Triangle Gardens, Salcedo Village, Makat...","Ayala Triangle Gardens, Salcedo Village, Makat...","Ayala Triangle Gardens, Salcedo Village, Makat...",121.023171,14.556196,"Filipino, American, Italian, Bakery",...,Botswana Pula(P),0,0,0,0,3,4.4,5BA829,Very Good,867
4,6315689,Bad Bird,162,Makati City,"Hole In The Wall, Floor 4, Century City Mall, ...","Century City Mall, Poblacion, Makati City","Century City Mall, Poblacion, Makati City, Mak...",121.027708,14.565899,American,...,Botswana Pula(P),0,0,0,0,3,4.4,5BA829,Very Good,858


### 1.1. Descrição de Colunas

#####  Column	     Description
#####  Restaurant ID	        ID do restaurante
#####  Restaurant Name	Nome do Restaurante
#####  Country Code	Código do País
#####  City	Nome da Cidade onde o restaurante está
#####  Address	Endereço do restaurante
#####  Locality	Localização e pontos de referência do restaurante
#####  Locality Verbose	Localização e pontos de referência do restaurante (Mais informações)
#####  Longitude	 Ponto geográfico de Longitude do Restaurante
#####  Latitude	Ponto geográfico de Latitude do Restaurante
#####  Cuisines	Tipos de Culinária servidos no restaurante
#####  Average Cost for two	 - Preço Médio de um prato para duas pessoas no restaurante
#####  Currency	- Moeda do país
#####  Has Table - booking	Se o restaurante possui serviços de reserva; 1 - Sim; 0 - Não
#####  Has Online delivery	Se o restaurante possui serviços de pedido on-line; 1 - Sim; 0 - Não
#####  Is delivering now	Se o restaurante faz entregas; 1 - Sim; 0 - Não
#####  Switch to order menu	-
#####  Price range	Variação de preços do restaurante; 1 a 4 - Quanto maior o valor, mais caro serão os pratos
#####  Aggregate rating	Nota média do restaurante
#####  Rating color	Código Hexadecimal da cor do restaurante com base em sua nota média
#####  Rating text	Categoria em que o restaurante está com base em sua nota média
#####  Votes	Quantidade de avaliações que o restaurante já recebeu

### 1.2 Renomeando Colunas

In [14]:
df2 = rename_columns(df2)
df2.head()


,restaurant_id,restaurant_name,country_code,city,address,locality,locality_verbose,longitude,latitude,cuisines,...,currency,has_table_booking,has_online_delivery,is_delivering_now,switch_to_order_menu,price_range,aggregate_rating,rating_color,rating_text,votes
0,6310675,Mama Lou's Italian Kitchen,162,Las Piñas City,"Block 1, Lot 36, Tropical Avenue Corner Tropic...",BF International,"BF International, Las Piñas City",121.009787,14.447615,Italian,...,Botswana Pula(P),1,0,0,0,3,4.6,3F7E00,Excellent,619
1,6310675,Mama Lou's Italian Kitchen,162,Las Piñas City,"Block 1, Lot 36, Tropical Avenue Corner Tropic...",BF International,"BF International, Las Piñas City",121.009787,14.447615,Italian,...,Botswana Pula(P),1,0,0,0,3,4.6,3F7E00,Excellent,619
2,6314542,Blackbird,162,Makati City,"Nielson Tower, Ayala Triangle Gardens, Salcedo...","Ayala Triangle Gardens, Salcedo Village, Makat...","Ayala Triangle Gardens, Salcedo Village, Makat...",121.024562,14.556042,"European, Asian",...,Botswana Pula(P),0,0,0,0,4,4.7,3F7E00,Excellent,469
3,6301293,Banapple,162,Makati City,"Ayala Triangle Gardens, Salcedo Village, Makat...","Ayala Triangle Gardens, Salcedo Village, Makat...","Ayala Triangle Gardens, Salcedo Village, Makat...",121.023171,14.556196,"Filipino, American, Italian, Bakery",...,Botswana Pula(P),0,0,0,0,3,4.4,5BA829,Very Good,867
4,6315689,Bad Bird,162,Makati City,"Hole In The Wall, Floor 4, Century City Mall, ...","Century City Mall, Poblacion, Makati City","Century City Mall, Poblacion, Makati City, Mak...",121.027708,14.565899,American,...,Botswana Pula(P),0,0,0,0,3,4.4,5BA829,Very Good,858


### 1.3. Dimensão dos Dados

In [15]:
df2.shape

(7527, 21)

### 1.4. Data Types

In [16]:
df2.dtypes

restaurant_id             int64
restaurant_name          object
country_code              int64
city                     object
address                  object
locality                 object
locality_verbose         object
longitude               float64
latitude                float64
cuisines                 object
average_cost_for_two      int64
currency                 object
has_table_booking         int64
has_online_delivery       int64
is_delivering_now         int64
switch_to_order_menu      int64
price_range               int64
aggregate_rating        float64
rating_color             object
rating_text              object
votes                     int64
dtype: object

### 1.5. Check NA

In [17]:
df2.isna().sum()

restaurant_id            0
restaurant_name          0
country_code             0
city                     0
address                  0
locality                 0
locality_verbose         0
longitude                0
latitude                 0
cuisines                15
average_cost_for_two     0
currency                 0
has_table_booking        0
has_online_delivery      0
is_delivering_now        0
switch_to_order_menu     0
price_range              0
aggregate_rating         0
rating_color             0
rating_text              0
votes                    0
dtype: int64

In [18]:
df2.loc[:, 'cuisines'].unique().tolist()

['Italian',
 'European, Asian',
 'Filipino, American, Italian, Bakery',
 'American',
 'Filipino',
 'Korean, Grill',
 'American, Filipino, Coffee',
 'Pizza, Italian',
 'Taiwanese',
 'Japanese, Latin American',
 'Japanese',
 'Italian, Pizza',
 'American, Filipino',
 'Japanese, Sushi',
 'Coffee, American, European',
 'Japanese, Ramen',
 'American, Italian, Bakery, Coffee',
 'Chinese',
 'Japanese, Korean',
 'Coffee',
 'European, Desserts',
 'European, Steak, Bakery, Desserts',
 'Filipino, Chinese',
 'Seafood, Filipino, Asian, European',
 'European, Asian, Indian',
 'Korean',
 'Filipino, Mexican',
 'American, Burger',
 'Singaporean, Street Food',
 'Vietnamese',
 'Latin American, Street Food',
 'Filipino, Asian',
 'Korean, Korean BBQ',
 'Filipino, Coffee',
 'European, Italian',
 'American, Asian',
 'European, American',
 'European, Filipino, American',
 'American, European',
 'Coffee, Japanese',
 'Japanese, Asian',
 'Seafood, American, Mediterranean, Japanese',
 'European, Filipino',
 'Healt

### 1.6. Treat NA

In [19]:
df2 = df2.dropna()
df2.isna().sum()

restaurant_id           0
restaurant_name         0
country_code            0
city                    0
address                 0
locality                0
locality_verbose        0
longitude               0
latitude                0
cuisines                0
average_cost_for_two    0
currency                0
has_table_booking       0
has_online_delivery     0
is_delivering_now       0
switch_to_order_menu    0
price_range             0
aggregate_rating        0
rating_color            0
rating_text             0
votes                   0
dtype: int64

### 1.7. Troca de Types

Não há necessidade de fazer a troca de tipos dos dados.

### 1.8. Estatística Descritiva

In [20]:
df2.describe()

,restaurant_id,country_code,longitude,latitude,average_cost_for_two,has_table_booking,has_online_delivery,is_delivering_now,switch_to_order_menu,price_range,aggregate_rating,votes
count,7.512000e+03,7512.000000,7512.000000,7512.000000,7.512000e+03,7512.000000,7512.000000,7512.000000,7512.0,7512.000000,7512.000000,7512.000000
mean,1.054327e+07,93.134718,33.763215,19.312227,7.165669e+03,0.062300,0.353168,0.174654,0.0,2.625399,4.122724,617.483360
std,7.075488e+06,99.026782,77.374736,23.274183,2.908967e+05,0.241716,0.477986,0.379696,0.0,0.959566,0.667171,1128.540817
min,5.490000e+02,1.000000,-122.700455,-41.330428,0.000000e+00,0.000000,0.000000,0.000000,0.0,1.000000,0.000000,0.000000
25%,3.500048e+06,1.000000,-4.257793,12.915952,6.000000e+01,0.000000,0.000000,0.000000,0.0,2.000000,4.000000,153.000000
50%,7.701329e+06,30.000000,73.782800,25.237157,2.900000e+02,0.000000,0.000000,0.000000,0.0,3.000000,4.200000,335.000000
75%,1.714709e+07,214.000000,79.833453,31.637156,6.000000e+02,0.000000,1.000000,0.000000,0.0,3.000000,4.500000,664.000000
max,1.904028e+07,216.000000,175.310552,55.976980,2.500002e+07,1.000000,1.000000,1.000000,0.0,4.000000,4.900000,41333.000000


# 2. Respostas

In [21]:
df1 = process_data(DATA_PATH)

## Problema de Negócio

Parabéns! Você acaba de ser contratado como Cientista de Dados da empresa
Fome Zero, e a sua principal tarefa nesse momento é ajudar o CEO Kleiton Guerra
a identificar pontos chaves da empresa, respondendo às perguntas que ele fizer
utilizando dados!
A empresa Fome Zero é uma marketplace de restaurantes. Ou seja, seu core
business é facilitar o encontro e negociações de clientes e restaurantes. Os
restaurantes fazem o cadastro dentro da plataforma da Fome Zero, que disponibiliza
informações como endereço, tipo de culinária servida, se possui reservas, se faz
entregas e também uma nota de avaliação dos serviços e produtos do restaurante,
dentre outras informações.


## Metodo SAPE

### Saída:
     - Resolver as perguntas de negócio e colocar as respostas em um dashbord no framework Streamlit.

### Processo:
     - Eu preciso baixar a base dados no kaggle, utilizar a estatística e o python para responder as perguntas de negócio e o streamlit para criação de um dashboard com o resultado das questões.

### Entrada:
     - Eu posso usar as funções necessárias do python para tratamento dos dados e resposta das perguntas e técnicas de design com bibliotecas do streamlit para criação do dashboard.

## Geral

### 1. Quantos restaurantes únicos estão registrados?

In [58]:
df1.loc[:, 'restaurant_id'].nunique()

6929

### 2. Quantos países únicos estão registrados?

In [52]:
df1['country_name'].nunique()

15

### 3. Quantas cidades únicas estão registradas?

In [53]:
df1['city'].nunique()

125

### 4. Qual o total de avaliações feitas?

In [54]:
df1_aux = df1['votes'].sum()

print('{:,.0f}'.format(df1_aux).replace(',', '.'))


4.194.533


### 5. Qual o total de tipos de culinária registrados?

In [55]:
df1['cuisines'].nunique()

165

In [79]:
paises = df1['country_name'].unique()

for pais in paises:
    print(pais)

Philippines
Brazil
Australia
United States of America
Canada
Singapure
United Arab Emirates
India
Indonesia
New Zeland
England
Qatar
South Africa
Sri Lanka
Turkey


## Pais

### 1. Qual o nome do país que possui mais cidades registradas?

In [74]:
df1_aux = (df1.loc[:, ['country_name', 'city']]
                    .groupby('country_name')
                    .nunique()
                    .sort_values('city', ascending=False)
                    .reset_index() )


# Plotando o gráfico
fig = px.bar(df1_aux, x='country_name', y='city', text='city', template='simple_white', labels={
   'city': 'Quantidade de Cidades', 
   'country_name': 'Países'
}  )

fig.show()


### 2. Qual o nome do país que possui mais restaurantes registrados?

In [73]:
df1_aux = df1.loc[:,['country_name','restaurant_id']].groupby(['country_name']).count().sort_values('restaurant_id', ascending=False).reset_index()


# Plotando o gráfico
fig = px.bar(df1_aux, x='country_name', y='restaurant_id', text_auto='restaurant_id', template='simple_white', labels={
   'restaurant_id': 'Quantidade de Restaurantes', 
   'country_name': 'Países'
} )
fig.update_traces(marker_line_color = 'rgb(8,48,107)', marker_line_width = 1.5 )

fig.show()

### 3. Qual o nome do país que possui mais restaurantes com o nível de preço igual a 4 registrados?

In [72]:

df1_aux = df1.loc[:, ['country_name','restaurant_name','price_type']].groupby(['country_name', 'restaurant_name']).count()

linhas_selecionadas=df1_aux['price_type']== 4

df1_aux = df1_aux.loc[linhas_selecionadas, :].reset_index().iloc[0,0]

df1_aux

'England'

### 4. Qual o nome do país que possui a maior quantidade de tipos de culinária distintos?

In [27]:
df1_aux = ( df1.loc[:, ['country_name', 'cuisines']]
                    .groupby('country_name')
                    .nunique()
                    .sort_values(by='cuisines', ascending=False)
                    .reset_index().iloc[0,0] )

df1_aux

'India'

### 5. Qual o nome do país que possui a maior quantidade de avaliações feitas?

In [25]:
df1.loc[:, [ 'votes', 'country_name' ]].groupby('country_name').sum().sort_values(by='votes', ascending=False).reset_index().iloc[0,0]


'India'

### 6. Qual o nome do país que possui a maior quantidade de restaurantes que fazem entrega?

In [65]:
df1_aux = df1.loc[df1['is_delivering_now'] == 1]

df1_aux.loc[:,['country_name', 'is_delivering_now']].groupby(['country_name']).count().sort_values(by='is_delivering_now', ascending=False).reset_index().iloc[0,0]

'India'

### 7. Qual o nome do país que possui a maior quantidade de restaurantes que aceitam reservas?

In [66]:
df1_aux = df1.loc[df1['has_table_booking'] == 1 ]

df1_aux.loc[:, ['country_name', 'has_table_booking']].groupby('country_name').count().sort_values(by='has_table_booking', ascending=False).reset_index().iloc[0,0]


'India'

### 8. Qual o nome do país que possui, na média, a maior quantidade de avaliações registrada?

In [64]:
df1_aux = ( df1.loc[:,['country_name','votes']]
                    .groupby('country_name')
                    .mean()
                    .sort_values(by='votes',ascending=False)
                    .reset_index()
                    .round(2) )


# Plotando o gráfico
fig = px.bar(df1_aux, x='country_name', y='votes', text='votes', template='simple_white', labels={
    'country_name': 'Países',
    'votes': 'Quantidade de Avaliações'
} )

fig.update_traces(marker_line_color = 'rgb(8,48,107)', marker_line_width = 1.5 )

fig.show()

### 9. Qual o nome do país que possui, na média, a maior nota média registrada?

In [28]:
df1.loc[:, ['aggregate_rating', 'country_name']].groupby('country_name').mean().sort_values('aggregate_rating', ascending=False).reset_index().iloc[0,0]


'Indonesia'

### 10. Qual o nome do país que possui, na média, a menor nota média registrada?

In [29]:
df1.loc[:, ['aggregate_rating', 'country_name']].groupby('country_name').mean().sort_values('aggregate_rating', ascending=True).reset_index().iloc[0,0]

'Brazil'

### 11. Qual a média de preço de um prato para dois por país?

In [63]:
df1_aux = (df1.loc[:, ['average_cost_for_two', 'country_name']]
                    .groupby('country_name')
                    .mean().sort_values('average_cost_for_two', ascending=False)
                    .reset_index()
                    .round(2) )
       

# Plotando o gráfico

fig = px.bar( df1_aux, x='country_name', y='average_cost_for_two', text='average_cost_for_two', template='simple_white', labels={
    'country_name': 'Países' ,
    'average_cost_for_two': 'Preço de prato para duas Pessoas'
} )

fig.update_traces(marker_line_color = 'rgb(8,48,107)', marker_line_width = 1.5 )
fig.show()

## Cidade

### 1. Qual o nome da cidade que possui mais restaurantes registrados?

In [61]:
df1_aux = ( df1.loc[:, ['country_name', 'city', 'restaurant_id']]
                    .groupby([ 'country_name','city' ])
                    .count()
                    .sort_values(['restaurant_id', 'city'], ascending=[False, True])
                    .reset_index().head(10) )

fig = px.bar(df1_aux, x='city', y='restaurant_id', 
                    text_auto= 'restaurant_id', color='country_name' ,labels= {
                        'country_name': 'Países',
                        'restaurant_id': 'Quantidade de Restaurantes',
                        'city': 'Cidade'
                    } , template='simple_white')
    
fig.update_layout(title_text='Top 10 Cidades com mais Restaurantes na Base de Dados', title_x=0.3)
fig.update_traces(marker_line_color = 'rgb(8,48,107)', marker_line_width = 1.5)   

fig.show()

### 2. Qual o nome da cidade que possui mais restaurantes com nota média acima de 4?



In [60]:
df1_aux = df1.loc[df1['aggregate_rating'] >= 4]

df2_aux = ( df1_aux.loc[:, ['country_name','city', 'aggregate_rating']]
                        .groupby(['city', 'country_name'])
                        .count()
                        .sort_values('aggregate_rating', ascending=False)
                        .reset_index()
                        .head(7) )

# Plotando o gráfico

fig = px.bar(df2_aux, x='city', y='aggregate_rating', 
text='aggregate_rating', color='country_name', template='simple_white', labels={
    'city': 'Cidade' ,
    'country_name': 'País' ,
    'aggregate_rating': 'Quantidade de Restaurantes'
} )
fig.update_traces(marker_line_color = 'rgb(8,48,107)', marker_line_width = 1.5 )

fig.show()


### 3. Qual o nome da cidade que possui mais restaurantes com nota média abaixo de 2.5?

In [59]:
df1_aux = df1.loc[df1['aggregate_rating'] <= 2.5]

df2_aux = ( df1_aux.loc[:, ['country_name','city', 'aggregate_rating']]
                        .groupby(['country_name', 'city' ])
                        .count()
                        .sort_values('aggregate_rating', ascending=False)
                        .reset_index()
                        .head(7) )

# Plotando o gráfico

fig = px.bar(df2_aux, x='city', y='aggregate_rating', 
text='aggregate_rating', color='country_name', template='plotly_white', labels={
    'city': 'Cidade' ,
    'country_name': 'País' ,
    'aggregate_rating': 'Quantidade de Restaurantes'
} )

fig.update_traces(marker_line_color = 'rgb(8,48,107)', marker_line_width = 1.5 )

fig.show()

### 4. Qual o nome da cidade que possui o maior valor médio de um prato para dois?

In [30]:
df1.loc[:, ['city', 'average_cost_for_two']].groupby('city').mean().sort_values('average_cost_for_two', ascending=False).reset_index().iloc[0,0]

'Adelaide'

### 5. Qual o nome da cidade que possui a maior quantidade de tipos de culinária distintas?

In [58]:

df1_aux = ( df1.loc[:, ['country_name','city', 'cuisines']]
                        .groupby(['country_name','city'])
                        .nunique()
                        .sort_values('cuisines', ascending=False)
                        .reset_index()
                        .head(10) )


# Plotando o gráfico
fig = px.bar(df1_aux, x='city', y='cuisines', 
text='cuisines', color='country_name', template='simple_white', labels={
    'city': 'Cidade' ,
    'country_name': 'País' ,
    'cuisines': 'Quantidade de Tipos Culinários Únicos'
} )

fig.update_traces(marker_line_color = 'rgb(8,48,107)', marker_line_width = 1.5 )

fig.show()


### 6. Qual o nome da cidade que possui a maior quantidade de restaurantes que fazem reservas?

In [34]:
df1_aux = df1.loc[df1['has_table_booking'] >= 1]

df2 =( df1_aux.loc[:, ['city', 'has_table_booking', 'country_name']]
                    .groupby(['city', 'country_name'])
                    .count()
                    .sort_values('has_table_booking', ascending=False)
                    .reset_index()
                    .iloc[0,0] )
df2

'Bangalore'

### 7. Qual o nome da cidade que possui a maior quantidade de restaurantes que fazem entregas?

In [36]:
df1_aux = df1.loc[df1['is_delivering_now'] >= 1]
df2 = (df1_aux.loc[:, ['city', 'is_delivering_now', 'country_name']]
                    .groupby(['city', 'country_name'])
                    .count()
                    .sort_values('is_delivering_now', ascending=False)
                    .reset_index()
                    .iloc[0,0] )
df2                   

'Vadodara'

### 8. Qual o nome da cidade que possui a maior quantidade de restaurantes que aceitam pedidos online?

In [37]:
df1_aux = df1.loc[df1['has_online_delivery'] >= 1]
df2 = ( df1_aux.loc[:, ['city', 'has_online_delivery', 'country_name']]
                    .groupby(['city', 'country_name'])
                    .count()
                    .sort_values('has_online_delivery', ascending=False)
                    .reset_index()
                    .iloc[0,0] )
df2

'Bhopal'

## Restaurantes

### 1. Qual o nome do restaurante que possui a maior quantidade de avaliações?

In [39]:
cols = ['restaurant_name', 'restaurant_id', 'votes', 'country_name', 'city', 'aggregate_rating', 'cuisines']

df1.loc[:, cols].sort_values(['votes', 'restaurant_id'], ascending=[False, True]).iloc[0,0]

'Bawarchi'

### 2. Qual o nome do restaurante com a maior nota média?

In [38]:
cols = ['restaurant_name', 'restaurant_id', 'votes', 'country_name', 'city', 'aggregate_rating', 'cuisines']

df1.loc[:, cols].sort_values(['aggregate_rating', 'restaurant_id'], ascending=[False, True]).head(20).iloc[0,0]

'Indian Grill Room'

### 3. Qual o nome do restaurante que possui o maior valor de uma prato para duas pessoas?

In [45]:
df1_aux = ( df1.loc[:,['restaurant_name', 'average_cost_for_two']]
                    .groupby('restaurant_name')
                    .max()
                    .sort_values('average_cost_for_two', ascending=False)
                    .reset_index().iloc[0,0] )
               
df1_aux

"d'Arry's Verandah Restaurant"

### 4. Qual o nome do restaurante de tipo de culinária brasileira que possui a menor média de avaliação?

In [44]:
country = df1.loc[df1['cuisines'] == 'Brazilian', :]

menor_mean_cuisines_brazilian = ( country.loc[:,['restaurant_name', 'cuisines', 'aggregate_rating']]
                     .groupby(['restaurant_name', 'cuisines'])
                     .max()
                     .sort_values('aggregate_rating', ascending=True)
                     .reset_index().iloc[0,0] )

menor_mean_cuisines_brazilian

'Quitéria'

### 5. Qual o nome do restaurante de tipo de culinária brasileira, e que é do Brasil, que possui a maior média de avaliação?

In [46]:
country = df1.loc[(df1['cuisines'] == 'Brazilian') & (df1['country_name'] == 'Brazil')]

maior_media_country_brazilian = ( country.loc[:,['restaurant_name','country_name', 'cuisines', 'aggregate_rating']]
                     .groupby(['restaurant_name', 'cuisines', 'country_name'])
                     .max()
                     .sort_values('aggregate_rating', ascending=False)
                     .reset_index().iloc[0,0] )

maior_media_country_brazilian

'Braseiro da Gávea'

### 6. Os restaurantes que aceitam pedido online são também, na média, os restaurantes que mais possuem avaliações registradas?

In [48]:
df_aux1 = ( df1.loc[:, [ 'has_online_delivery', 'votes']]
                    .groupby(['has_online_delivery'])  
                    .mean().sort_values('votes', ascending=False)
                    .reset_index() )
df_aux1

,has_online_delivery,votes
0,1,838.821664
1,0,479.421018


### 7. Os restaurantes que fazem reservas são também, na média, os restaurantes que possuem o maior valor médio de um prato para duas pessoas?

In [147]:


df_aux1 = ( df1.loc[:, [ 'has_table_booking', 'average_cost_for_two']]
                    .groupby([ 'has_table_booking'])
                    .mean()
                    .sort_values('average_cost_for_two', ascending=False)
                    .reset_index() )
df_aux1

,has_table_booking,average_cost_for_two
0,1,69998.423810
1,0,3488.596866


### 8. Os restaurantes do tipo de culinária japonesa dos Estados Unidos da América possuem um valor médio de prato para duas pessoas maior que as churrascarias americanas (BBQ)?

In [155]:
country = (df1['country_name'] == 'United States of America') & ((df1['cuisines'] == 'BBQ') | (df1['cuisines'] == 'Japanese'))

maior_media_country_bbq = ( df1.loc[country,['cuisines', 'average_cost_for_two']]
                     .groupby(['cuisines'])
                     .mean()
                     .sort_values('average_cost_for_two', ascending=False)
                     .reset_index() )

maior_media_country_bbq





,cuisines,average_cost_for_two
0,Japanese,56.406250
1,BBQ,39.642857


## Tipos de Culinária


### 1. Dos restaurantes que possuem o tipo de culinária italiana, qual o nome do restaurante com a maior média de avaliação?

In [120]:
cols = [
    'restaurant_id', 'restaurant_name', 'country_name', 
    'city', 'cuisines', 'average_cost_for_two', 
    'aggregate_rating', 'votes'
]

lines = (df1['cuisines'] == 'Italian')

df1.loc[lines, cols].sort_values(['aggregate_rating', 'restaurant_id'], ascending=[False, True]).iloc[0,1]

'Darshan'

### 2. Dos restaurantes que possuem o tipo de culinária italiana, qual o nome do restaurante com a menor média de avaliação?

In [164]:
cols = [
    'restaurant_id', 'restaurant_name', 'country_name', 
    'city', 'cuisines', 'average_cost_for_two', 
    'aggregate_rating', 'votes'
]

lines = (df1['cuisines'] == 'Italian')

df1.loc[lines, cols].sort_values(['aggregate_rating', 'restaurant_id'], ascending=[True, True]).iloc[0,1]

'Avenida Paulista'

### 3. Dos restaurantes que possuem o tipo de culinária americana, qual o nome do restaurante com a maior média de avaliação?

In [121]:
cols = [
    'restaurant_id', 'restaurant_name', 'country_name', 
    'city', 'cuisines', 'average_cost_for_two', 
    'aggregate_rating', 'votes'
]

lines = (df1['cuisines'] == 'American')

df1.loc[lines, cols].sort_values(['aggregate_rating', 'restaurant_id'], ascending=[False, True]).iloc[0, 1]

'Burger & Lobster'

### 4. Dos restaurantes que possuem o tipo de culinária americana, qual o nome do restaurante com a menor média de avaliação?

In [169]:
cols = [
    'restaurant_id', 'restaurant_name', 'country_name', 
    'city', 'cuisines', 'average_cost_for_two', 
    'aggregate_rating', 'votes'
]

lines = (df1['cuisines'] == 'American')

df1.loc[lines, cols].sort_values(['aggregate_rating', 'restaurant_id'], ascending=[True, True]).iloc[0, 1]

'Alston Bar & Beef'

### 5. Dos restaurantes que possuem o tipo de culinária árabe, qual o nome do restaurante com a maior média de avaliação?

In [49]:
df2 = df1.loc[df1['cuisines'] == 'Arabian', :]

df_aux1 = ( df2.loc[:, ['restaurant_name', 'cuisines', 'aggregate_rating']]
                    .groupby(['restaurant_name', 'cuisines'])
                    .max()
                    .sort_values('aggregate_rating', ascending=False)
                    .reset_index().iloc[0,0])
df_aux1

'Mandi@36'

### 6. Dos restaurantes que possuem o tipo de culinária árabe, qual o nome do restaurante com a menor média de avaliação?

In [192]:
df2 = df1.loc[df1['cuisines'] == 'Arabian', :]

df_aux1 = ( df2.loc[:, ['restaurant_name', 'cuisines', 'aggregate_rating']]
                    .groupby(['restaurant_name', 'cuisines'])
                    .min()
                    .sort_values('aggregate_rating', ascending=True)
                    .reset_index().iloc[0,0] )
df_aux1

'Raful'

### 7. Dos restaurantes que possuem o tipo de culinária japonesa, qual o nome do restaurante com a maior média de avaliação?

In [176]:
cols = [
    'restaurant_id', 'restaurant_name', 'country_name', 
    'city', 'cuisines', 'average_cost_for_two', 
    'aggregate_rating', 'votes'
]

lines = (df1['cuisines'] == 'Japanese')

df1.loc[lines, cols].sort_values(['aggregate_rating', 'restaurant_id'], ascending=[False, True]).iloc[0, 1]

'Sushi Samba'

### 8. Dos restaurantes que possuem o tipo de culinária japonesa, qual o nome do restaurante com a menor média de avaliação?

In [190]:
df2 = df1.loc[df1['cuisines'] == 'Japanese', :]

df_aux1 = ( df2.loc[:, ['restaurant_name', 'cuisines', 'aggregate_rating']]
                    .groupby(['restaurant_name', 'cuisines'])
                    .min()
                    .sort_values('aggregate_rating', ascending=True)
                    .reset_index().iloc[0,0] )
df_aux1

'Banzai Sushi'

### 9. Dos restaurantes que possuem o tipo de culinária caseira, qual o nome do restaurante com a maior média de avaliação?

In [189]:
df2 = df1.loc[df1['cuisines'] == 'Home-made', :]

df_aux1 = ( df2.loc[:, ['restaurant_name', 'cuisines', 'aggregate_rating']]
                    .groupby(['restaurant_name', 'cuisines'])
                    .max()
                    .sort_values('aggregate_rating', ascending=False)
                    .reset_index().iloc[0,0] )
df_aux1

'Kanaat Lokantası'

### 10. Dos restaurantes que possuem o tipo de culinária caseira, qual o nome do restaurante com a menor média de avaliação?

In [188]:
df2 = df1.loc[df1['cuisines'] == 'Home-made', :]

df_aux1 = ( df2.loc[:, ['restaurant_name', 'cuisines', 'aggregate_rating']]
                    .groupby(['restaurant_name', 'cuisines'])
                    .min()
                    .sort_values('aggregate_rating', ascending=True)
                    .reset_index().iloc[0,0] )
df_aux1

'GurMekan Restaurant'

### 11. Qual o tipo de culinária que possui o maior valor médio de um prato para duas pessoas?

In [184]:

df_aux1 = ( df1.loc[:, ['cuisines', 'average_cost_for_two']]
                    .groupby(['cuisines'])
                    .max()
                    .sort_values('average_cost_for_two', ascending=False)
                    .reset_index().head(1) )
df_aux1

,cuisines,average_cost_for_two
0,Modern Australian,25000017


### 12. Qual o tipo de culinária que possui a maior nota média?

In [56]:
df_aux1 = ( df1.loc[:, ['cuisines', 'aggregate_rating']]
                    .groupby(['cuisines'])
                    .mean()
                    .sort_values('aggregate_rating', ascending=False)
                    .reset_index().head(20).round(2) )

#  Plotando o gráfico
fig = px.bar( df_aux1, x='cuisines', y='aggregate_rating', text='aggregate_rating', template='plotly_dark', labels={
    'cuisines': 'Tipo de Culinária',
    'aggregate_rating': 'Média da Avaliação Média'
} )
fig.update_traces(marker_line_color = 'rgb(8,48,107)', marker_line_width = 1.5)

fig.show()



In [57]:
df_aux1 = ( df1.loc[:, ['cuisines', 'aggregate_rating']]
                    .groupby(['cuisines'])
                    .mean()
                    .sort_values('aggregate_rating', ascending=True)
                    .reset_index().head(20).round(2) )

#  Plotando o gráfico
fig = px.bar( df_aux1, x='cuisines', y='aggregate_rating', text='aggregate_rating', template='plotly_dark', labels={
    'cuisines': 'Tipo de Culinária',
    'aggregate_rating': 'Média da Avaliação Média'
} )
fig.update_traces(marker_line_color = 'rgb(8,48,107)', marker_line_width = 1.5)

fig.show()


### 13. Qual o tipo de culinária que possui mais restaurantes que aceitam pedidos online e fazem entregas?

In [53]:
df_aux1 = ( df1.loc[:, ['cuisines', 'has_online_delivery']]
                    .groupby(['cuisines'])
                    .count()
                    .sort_values('has_online_delivery', ascending=False)
                    .reset_index().head(1) )
df_aux1

,cuisines,has_online_delivery
0,North Indian,878
